In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import matplotlib
import sklearn
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import tensorflow as tf

from os import makedirs
import os.path

from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import AveragePooling1D
from tensorflow.keras.layers import add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model

import warnings

warnings.filterwarnings('ignore')

In [ ]:
retval = os.getcwd()
print ("当前工作目录为 %s" %retval)
os.chdir("/content/gdrive/My Drive/iss/prumls")
workpath = os.getcwd()
print ("修改后工作目录为 %s" %workpath)

当前工作目录为 /content/gdrive/My Drive/iss/prumls
修改后工作目录为 /content/gdrive/My Drive/iss/prumls


In [ ]:
#Loading the dataset and cleaning the data
diabetes_data = pd.read_csv('/content/gdrive/My Drive/iss/prumls/data/diabetes.csv')

var_list = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
diabetes_data[var_list] = diabetes_data[var_list].replace(0,np.NaN)

def find_median(var):
    temp = diabetes_data[diabetes_data[var].notnull()][[var,'Outcome']]
    temp = temp.groupby('Outcome').median().reset_index()
    return temp[var].values

for var in var_list:
    diabetes_data.loc[(diabetes_data['Outcome'] == 0) & (diabetes_data[var].isnull()), var] = find_median(var)[0]
    diabetes_data.loc[(diabetes_data['Outcome'] == 1) & (diabetes_data[var].isnull()), var] = find_median(var)[1]


In [ ]:
total = diabetes_data.isnull().sum().sort_values(ascending=False)
percent = (diabetes_data.isnull().sum()/diabetes_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

,Total,Percent
Outcome,0,0.0
Age,0,0.0
DiabetesPedigreeFunction,0,0.0
BMI,0,0.0
Insulin,0,0.0


In [ ]:
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split

X = minmax_scale(diabetes_data.drop("Outcome",axis = 1).values)

outcome_ = diabetes_data.Outcome.values
y = np.zeros([outcome_.shape[0],1], dtype=np.int64)
    
for l in range(y.shape[0]):
   y[l,0] = outcome_[l]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=1/3,random_state=42, stratify=y)
print(X_train.shape,y_train.shape)

(512, 8) (512, 1)


In [ ]:
def makeSteps(dat, length, dist):
    width           = dat.shape[1]
    numOfSteps      = int(np.floor((width-length)/dist)+1)
    
                                        # Initialize the output
    segments        = np.zeros([dat.shape[0],numOfSteps,length],
                               dtype=dat.dtype)
    
    for l in range(numOfSteps):
        segments[:,l,:]     = dat[:,(l*dist):(l*dist+length)]
        
    return segments

length = 4
dist = 1
trDat = makeSteps(X_train, length, dist)
tsDat = makeSteps(X_test, length, dist)

trLbl = y_train
tsLbl = y_test

trLbl           = to_categorical(trLbl)
tsLbl           = to_categorical(tsLbl)
                               
num_classes     = tsLbl.shape[1]

print('The shape of trDat is', trDat.shape, "and the type is", trDat.dtype)
print('The shape of trLbl is', trLbl.shape, "and the type is", trLbl.dtype)
print('')
print('The shape of tsDat is', tsDat.shape, "and the type is", tsDat.dtype)
print('The shape of tsLbl is', tsLbl.shape, "and the type is", tsLbl.dtype)

The shape of trDat is (512, 5, 4) and the type is float64
The shape of trLbl is (512, 2) and the type is float32

The shape of tsDat is (256, 5, 4) and the type is float64
The shape of tsLbl is (256, 2) and the type is float32


#prepare the models if needed

In [ ]:
def lrSchedule(epoch):
    lr  = 1e-3
    
    if epoch > 160:
        lr  *= 0.5e-3        
    elif epoch > 140:
        lr  *= 1e-3       
    elif epoch > 120:
        lr  *= 1e-2     
    elif epoch > 80:
        lr  *= 1e-1
        
    print('Learning rate: ', lr)
    return lr

LRScheduler     = LearningRateScheduler(lrSchedule)

#                                                                                 # Step 2
# folderpath      = '/content/gdrive/My Drive/iss/prumls/data/'
# filepath        = folderpath + modelname + ".hdf5"
# checkpoint      = ModelCheckpoint(filepath, 
#                                   monitor='val_accuracy', 
#                                   verbose=0, 
#                                   save_best_only=True, 
#                                   mode='max')

# csv_logger      = CSVLogger(folderpath+modelname +'.csv')                       # Step 2
# callbacks_list  = [checkpoint,csv_logger,LRScheduler]                           # Step 3

# print("Callbacks created:")
# print(callbacks_list[0])
# print(callbacks_list[1])
# print(callbacks_list[2])
# print('')
# print("Path to model:", filepath)
# print("Path to log:  ", folderpath+modelname+'.csv')

In [ ]:

# fit model on dataset 
from tensorflow.keras import optimizers

optmz       = optimizers.Adam(lr=0.001)                     
                                               

                                                                                # Step 3
def resLyr(inputs,
           numFilters=16,
           kernelSz=3,
           strides=1,
           activation='relu',
           batchNorm=True,
           convFirst=True,
           lyrName=None):

    convLyr     = Conv1D(numFilters,
                         kernel_size=kernelSz,
                         strides=strides,
                         padding='same',
                         kernel_initializer='he_normal',
                         kernel_regularizer=l2(1e-4),
                         name=lyrName+'_conv' if lyrName else None)
    x           = inputs
    
    if convFirst:
        x       = convLyr(x)
        
        if batchNorm:
            x   = BatchNormalization(name=lyrName+'_bn' if lyrName else None)(x)
            
        if activation is not None:
            x   = Activation(activation,
                             name=lyrName+'_'+activation if lyrName else None)(x)
    else:
        if batchNorm:
            x   = BatchNormalization(name=lyrName+'_bn' if lyrName else None)(x)
            
        if activation is not None:
            x   = Activation(activation,
                             name=lyrName+'_'+activation if lyrName else None)(x)
            
        x       = convLyr(x)
    return x

                                                                                # Step 4
def resBlkV1(inputs,
             numFilters=16,
             numBlocks=3,
             downsampleOnFirst=True,
             names=None):
    
    x       = inputs
    
    for run in range(0,numBlocks):
        strides = 1
        blkStr  = str(run+1)
        
        if downsampleOnFirst and run == 0:
            strides     = 2
            
        y       = resLyr(inputs=x,
                         numFilters=numFilters,
                         strides=strides,
                         lyrName=names+'_Blk'+blkStr+'_Res1' if names else None)
        y       = resLyr(inputs=y,
                         numFilters=numFilters,
                         activation=None,
                         lyrName=names+'_Blk'+blkStr+'_Res2' if names else None)
        
        if downsampleOnFirst and run == 0:
            x   = resLyr(inputs=x,
                         numFilters=numFilters,
                         kernelSz=1,
                         strides=strides,
                         activation=None,
                         batchNorm=False,
                         lyrName=names+'_Blk'+blkStr+'_lin' if names else None)

        x       = add([x,y],
                      name=names+'_Blk'+blkStr+'_add' if names else None)
        x       = Activation('relu',
                             name=names+'_Blk'+blkStr+'_relu' if names else None)(x)
        
    return x
    
                                                                                # Step 5
def createResNetV1(inputShape=(5, 4),
                   numClasses=2):
    inputs = Input(shape=inputShape)
    v = resLyr(inputs)
    v = resBlkV1(inputs=v, 
                 numFilters=16, 
                 numBlocks=3, 
                 downsampleOnFirst=True
                 )
    v = resBlkV1(inputs=v,
                 numFilters=16,
                 numBlocks=3,
                 downsampleOnFirst=False
                 )
    v = resBlkV1(inputs=v,
                 numFilters=16,
                 numBlocks=3,
                 downsampleOnFirst=True
                 )
    v = AveragePooling1D(pool_size=2)(v)
    v = Flatten()(v)
    outputs = Dense(num_classes, 
              activation='softmax', 
              kernel_initializer='he_normal')(v)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',
                  optimizer=optmz,
                  metrics=['accuracy'])
    #fit model
    model.fit(trDat, trLbl, batch_size=16, 
                    validation_data=(tsDat, tsLbl), 
                    epochs=100, 
                    verbose=1,
                    steps_per_epoch=len(trDat)/16
                    )  
    return model
                                                                    
#makedirs('resNetModels')
if not os.path.exists('resNetModels'):
    os.makedirs('resNetModels')
n_members = 2
for i in range(n_members):
    # fit model
    model = createResNetV1()
    # save model
    filename = 'resNetModels/resNet_' + str(i + 1) + '.h5'
    model.save(filename)
    print('>Saved %s' % filename)

Epoch 1/100
32/32 [==============================] - 1s 30ms/step - loss: 1.3971 - accuracy: 0.6641 - val_loss: 1.1021 - val_accuracy: 0.5625
Epoch 2/100
32/32 [==============================] - 0s 8ms/step - loss: 0.6252 - accuracy: 0.7520 - val_loss: 0.8506 - val_accuracy: 0.4648
Epoch 3/100
32/32 [==============================] - 0s 9ms/step - loss: 0.5227 - accuracy: 0.7969 - val_loss: 0.8715 - val_accuracy: 0.4414
Epoch 4/100
32/32 [==============================] - 0s 9ms/step - loss: 0.4872 - accuracy: 0.8203 - val_loss: 0.9190 - val_accuracy: 0.4648
Epoch 5/100
32/32 [==============================] - 0s 9ms/step - loss: 0.4729 - accuracy: 0.8223 - val_loss: 0.8744 - val_accuracy: 0.5078
Epoch 6/100
32/32 [==============================] - 0s 8ms/step - loss: 0.4417 - accuracy: 0.8262 - val_loss: 0.8103 - val_accuracy: 0.5898
Epoch 7/100
32/32 [==============================] - 0s 8ms/step - loss: 0.4193 - accuracy: 0.8535 - val_loss: 0.7835 - val_accuracy: 0.5977
Epoch 8/100


In [ ]:
#seperate stack
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack

In [ ]:
# load models from file
def load_resNet_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'resNetModels/resNet_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models
 
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX
 
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    model = LogisticRegression()
    model.fit(stackedX, inputy)
    return model
 
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat
 

In [ ]:
# load all models
n_members = 3
members = load_resNet_models(n_members)
print('Loaded %d models' % len(members))
# evaluate standalone models on test dataset
for model in members:
    # testy_enc = to_categorical(tsLbl)
    _, acc = model.evaluate(tsDat, tsLbl, verbose=0)
    print('Model Accuracy: %.3f' % acc)


>loaded resNetModels/resNet_1.h5
>loaded resNetModels/resNet_2.h5
>loaded resNetModels/resNet_3.h5
Loaded 3 models
Model Accuracy: 0.773
Model Accuracy: 0.812
Model Accuracy: 0.840


In [ ]:
# fit stacked model using the ensemble
model = fit_stacked_model(members, tsDat, y_test)
# evaluate model on test set
yhat = stacked_prediction(members, model, tsDat)
acc = accuracy_score(y_test, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

Stacked Test Accuracy: 0.840
